In [ ]:
from Simulations import *
import os
import pickle
from joblib import Parallel, delayed
import dill
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
save_plots = True

In [ ]:
def calculate_mcc(TP, TN, FP, FN):
    return np.divide((TP * TN - FP * FN),np.sqrt((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN)))

# Load results & example MetaExperiments

In [ ]:
meta_experiment1 = dill.load(open(r"example_1.p","rb"))
meta_experiment2 = dill.load(open(r"example_2.p","rb"))

In [ ]:
result = []
for index, file in enumerate(os.listdir(r"Result")): 
    meta_experiment = dill.load(open(r"Result\{}".format(file),"rb"))
    meta_experiment['simulation_nr'] = index+1
    result.append(meta_experiment)
    
long_df = pd.concat(result)
long_df['n_simulations'] = long_df['simulation_nr'].max()

In [ ]:
long_df.groupby(['policy']).sum()

In [ ]:
relative_regret = long_df.groupby(['policy']).mean()[['total_reward','regret','sample_complexity']]

In [ ]:
relative_regret

In [ ]:
relative_regret = long_df.groupby(['policy']).mean()[['total_reward','regret','sample_complexity']]

relative_regret['n_arms'] = df_sim.groupby(['policy'])['n_experiments'].mean()

relative_regret = relative_regret.round(2)
relative_regret = relative_regret.rename(columns={'total_reward':'Average total reward',
                                                  'regret': 'Average regret',
                                                  'n_arms': 'Average number of arms evaluated'})

relative_regret

In [ ]:
f = open(r"C:\Building Blocks\Innovation - Documents\129. Thesis Jop\Thesis Jop Heuvelmans - BB & JADS\Figures\relative_regret.tex","w+")
f.write(relative_regret.to_latex())
f.close()

In [ ]:
df_sim = long_df.groupby(['policy','alpha','initial_pulls','simulation_nr']).agg({'regret':'mean',
                                   'net_regret':'mean',
                                   'total_reward': 'mean',
                                   'sample_complexity': 'mean',
                                   'TP': 'sum',
                                   'TN': 'sum',
                                   'FP': 'sum',
                                   'FN': 'sum',
                                   'n_simulations':'mean'}).reset_index()

df_sim['n_experiments'] = (df_sim['TP'] + df_sim['TN'] + df_sim['FP'] + df_sim['FN'])
df_sim['n_variants'] = 4 + (df_sim['n_experiments']-1) * (3)
df_sim['MCC'] = df_sim.apply(lambda x: calculate_mcc(x['TP'], x['TN'], x['FP'], x['FN']),axis=1)
df_sim['recall'] = df_sim['TP']/(df_sim['TP']+df_sim['FN'])
df_sim['precision'] = df_sim['TP']/(df_sim['TP'] + df_sim['FP'])
df_sim['FPR'] = df_sim['FP']/(df_sim['FP']+df_sim['TN'])
df_sim['FNR'] = df_sim['FN']/(df_sim['FN']+df_sim['TP'])
df_sim['FDR'] = df_sim['FP']/(df_sim['FP']+df_sim['TP'])

In [ ]:
df = df_sim.groupby(['policy','alpha','initial_pulls']).agg('mean').reset_index()
#del long_df
#del df_sim

In [ ]:
#Fix the colors
policies = ['Bootstrapped LUCB', 'TTTS', 'A/B testing','LUCB']
plt_colors = plt.rcParams['axes.prop_cycle'].by_key()['color'][:4]
colors = {policy: color for policy, color in zip(policies, plt_colors)}
colors['MCC'] = plt.rcParams['axes.prop_cycle'].by_key()['color'][4]
colors['precision'] = plt.rcParams['axes.prop_cycle'].by_key()['color'][5]
colors['recall'] = plt.rcParams['axes.prop_cycle'].by_key()['color'][6]
colors['FDR'] = plt.rcParams['axes.prop_cycle'].by_key()['color'][7]
colors['mean'] = plt.rcParams['axes.prop_cycle'].by_key()['color'][8]
colors['alpha'] = plt.rcParams['axes.prop_cycle'].by_key()['color'][9]

# Example of variant queue

In [ ]:
plt.figure(figsize=(10,5))
plt.scatter(np.arange(len(meta_experiment1.variant_queue)), meta_experiment1.variant_queue, label='Success probability (p)')
plt.xlabel('Variant number')
plt.ylabel('Success probability (p)')
#plt.title('Example of a variant_queue')
plt.legend(loc='upper left')
if save_plots: plt.savefig(r"C:\Building Blocks\Innovation - Documents\129. Thesis Jop\Thesis Jop Heuvelmans - BB & JADS\Figures\variant_queue.pdf")
plt.show()

# Results to table

In [ ]:
clean_results = df[['policy','alpha','regret','MCC','TP','TN','FP','FN','n_variants']]
clean_results = clean_results.round(3)
clean_results.head()

In [ ]:
if save_plots: 
    f = open(r"C:\Building Blocks\Innovation - Documents\129. Thesis Jop\Thesis Jop Heuvelmans - BB & JADS\Figures\results1.tex","w+")
    f.write(clean_results.to_latex())
    f.close()

# Scatter plot

In [ ]:
groups = df.groupby(['policy'])
plt.figure(figsize=(10,7))
for name, group in groups:
    plt.scatter(group['MCC'], group['regret'], label=name, color=colors[name])
#df.apply(lambda row: plt.text(row['MCC']*1.001, row['total_reward']*1.001, round(row['alpha'],3)), axis=1)
plt.legend(loc='upper left')
plt.xlabel('MCC (Accuracy)')
plt.ylabel('Regret (Efficiency)')
#plt.ylim(390000, 480000)
plt.xlim(0.1, 0.85)
if save_plots: plt.savefig(r"C:\Building Blocks\Innovation - Documents\129. Thesis Jop\Thesis Jop Heuvelmans - BB & JADS\Figures\scatter.pdf")
plt.show()

# Top 15 Accuracy

In [ ]:
top15_accuracy = df.sort_values(['MCC'], ascending=False).head(15)[['policy','alpha','MCC','initial_pulls']].reset_index().drop(['index'],axis=1)
top15_accuracy.index += 1
top15_accuracy

In [ ]:
#Write top 15 in accuracy to LaTex table
if save_plots: 
    f = open(r"C:\Building Blocks\Innovation - Documents\129. Thesis Jop\Thesis Jop Heuvelmans - BB & JADS\Figures\accuracy_top15.tex","w+")
    f.write(top15_accuracy.to_latex())
    f.close()

# MCC, precision and recall

In [ ]:
groups = df.groupby(['policy'])
fig, axs = plt.subplots(3, 1, figsize=(10,14))
metrics = [('a','MCC'), ('b','precision'), ('c','recall')]
for ax, metric in zip(axs.flat, metrics): 
    for name, group in groups:
        ax.plot(group['alpha'], group[metric[1]], label=name, color=colors[name])
        ax.set_xlabel('alpha')
        ax.set_title('({}): {}'.format(metric[0], metric[1]))
        ax.legend(loc='lower right')
        #ax.set_ylim(0.5,1)
fig.tight_layout(pad=2.0)
plt.savefig(r"C:\Building Blocks\Innovation - Documents\129. Thesis Jop\Thesis Jop Heuvelmans - BB & JADS\Figures\accuracy_metrics.pdf")
plt.show()

# Alpha & FPR

In [ ]:
df['alpha/(K*2)'] = df['alpha'] / (4*2)

groups = df.groupby(['policy'])
fig, axs = plt.subplots(1, 1, figsize=(10,5))
for name, group in groups:
    axs.plot(group['alpha'], group['FPR'], label='FPR {}'.format(name), color=colors[name])       
axs.set_title('FPR')
#axs.set_ylim(0, 0.7)
axs.plot(group['alpha'], group['alpha'], label='alpha', color=colors['precision'])
axs.plot(group['alpha'], group['alpha/(K*2)'], label='alpha/(K*2)', linestyle=':', color=colors['precision'])
axs.legend()
axs.set_ylabel('alpha, FPR')
axs.set_xlabel('alpha')

fig.tight_layout(pad=2.0)

plt.savefig(r"C:\Building Blocks\Innovation - Documents\129. Thesis Jop\Thesis Jop Heuvelmans - BB & JADS\Figures\fpr.pdf")
plt.show()

# Net regret and sample size

In [ ]:
groups = df.groupby(['policy'])
plt.figure(figsize=(10,7))
for name, group in groups:
    plt.scatter(group['sample_complexity'], group['regret'], label=name, color=colors[name])
#df.apply(lambda row: plt.text(row['MCC']*1.001, row['total_reward']*1.001, round(row['alpha'],3)), axis=1)
plt.legend(loc='upper left')
plt.xlabel('Sample complexity')
plt.ylabel('Regret')
#plt.title('Regret and sample complexity')
#plt.ylim(390000, 480000)
#plt.xlim(0.1, 0.9)
plt.savefig(r"C:\Building Blocks\Innovation - Documents\129. Thesis Jop\Thesis Jop Heuvelmans - BB & JADS\Figures\regret_complexity.pdf")
plt.show()

# Example LUCB

In [ ]:
def phi(n_pulls, delta):
    """ Calculates phi to use in confidence interval calculations.

    :param n_pulls int: number of pulls taken from this arm.
    :param delta float: confidence level.

    :return: phi float: phi.
    """
    item_1 = np.log(1 / delta)
    item_2 = 3 * np.log(np.log(1 / delta))
    item_3 = (3 / 2) * np.log(np.log(np.e * n_pulls))
    return np.sqrt((item_1 + item_2 + item_3) / n_pulls)

In [ ]:
high_rewards = [np.random.binomial(1,0.2,1)[0] for i in range(25000)]
high_mean = [np.mean(high_rewards[0:i]) for i in range(len(high_rewards))]
high_LCB = [(high_mean[i] - phi(i, (0.05 / (2 * 4)))) for i in range(len(high_rewards))]
high_UCB = [(high_mean[i] + phi(i, (0.05 / 2))) for i in range(len(high_rewards))]

In [ ]:
low_rewards = [np.random.binomial(1,0.15,1)[0] for i in range(25000)]
low_mean = [np.mean(low_rewards[0:i]) for i in range(len(low_rewards))]
low_LCB = [(low_mean[i] - phi(i, (0.05 / (2 * 4)))) for i in range(len(low_rewards))]
low_UCB = [(low_mean[i] + phi(i, (0.05 / 2))) for i in range(len(low_rewards))]

In [ ]:
for i in range(len(low_LCB)):
    if  high_LCB[i] > low_UCB[i]: 
        cross_over = i
        break
fig, ax = plt.subplots(figsize=(10,6))
ax.plot(np.arange(25000),high_mean, color='blue', label='true sc = 0.2')
ax.fill_between(np.arange(25000), high_LCB, high_UCB)
ax.plot(np.arange(25000),low_mean, color='red', label='true mean = 0.1')
ax.fill_between(np.arange(25000), low_LCB, low_UCB)
ax.vlines(cross_over, 0, 0.5, linestyle=':', label='Terminate')
ax.legend()
#plt.plot(np.arange(10000),LCB, color='blue', line)
#plt.plot(np.arange(10000),UCB, color='blue')
#plt.vlines(2500, -3, 3, linestyle=':')
ax.set_ylabel('reward')
ax.set_xlabel('number of pulls')
ax.set_ylim(0, 0.5)
if save_fig: plt.savefig(r"C:\Building Blocks\Innovation - Documents\129. Thesis Jop\Thesis Jop Heuvelmans - BB & JADS\Figures\LUCB.pdf")
plt.show()

# Example LUCB

In [ ]:
save_fig = True
arm_names = {0: 'A',
             1: 'B',
             2: 'C',
             3: 'D'}

meta_experiment = pickle.load(open(r"example_3.p","rb"))

experiment_number = 0
BS_LUCB = meta_experiment.simulations[0].experiments[experiment_number]
LUCB = meta_experiment.simulations[1].experiments[experiment_number]

colours = ["#9b59b6", "#e74c3c", "#34495e", "#2ecc71"]

initial_pulls = 2500
pulls_per_round = 100 

initial_pulls = 2500
pulls_per_round = 100 
max_pulls = 100000

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2,2,figsize=(11,8))
for index, arm in enumerate(BS_LUCB.arms):
    ax1.plot([n_pulls for n_pulls, value in arm.mean_history] + [BS_LUCB.total_pulls],
             [value for n_pulls, value in arm.mean_history] + [arm.mean_history[-1][1]],
             color = colours[index],
             label='mean arm {}'.format(arm_names[arm.identifier]))
    ax1.fill_between([n_pulls for n_pulls, value in arm.mean_history] + [BS_LUCB.total_pulls],
                     [value for n_pulls, value in arm.LCB_history] + [arm.LCB_history[-1][1]],
                     [value for n_pulls, value in arm.UCB_history] + [arm.UCB_history[-1][1]],
                     alpha=0.5,
                     color = colours[index])
    ax1.hlines(arm.true_mean, 0,BS_LUCB.total_pulls, linestyle=':', color = colours[index], label='true success probability arm {}'.format(arm_names[arm.identifier]))
ax1.vlines(initial_pulls*len(BS_LUCB.arms), 0.09,0.175, linestyle='--')
ax1.set_xlim(0,max_pulls)
ax1.text((initial_pulls*len(BS_LUCB.arms))*1.1, 0.17, 'initial_pulls done')
ax1.set_ylim(0.09,0.175)
ax1.set_title('(a) Mean and CB for Bootstrapped LUCB')


for index, arm in enumerate(LUCB.arms):
    ax2.plot([n_pulls for n_pulls, value in arm.mean_history] + [LUCB.total_pulls],
              [value for n_pulls, value in arm.mean_history] + [arm.mean_history[-1][1]],
              color = colours[index])
    ax2.fill_between([n_pulls for n_pulls, value in arm.mean_history] + [LUCB.total_pulls],
                     [value for n_pulls, value in arm.LCB_history] + [arm.LCB_history[-1][1]],
                     [value for n_pulls, value in arm.UCB_history] + [arm.UCB_history[-1][1]],
                     alpha=0.5,
                     color = colours[index])
    ax2.hlines(arm.true_mean, 0,LUCB.total_pulls, linestyle=':', color = colours[index])
ax2.set_xlim(0, max_pulls)
ax2.set_ylim(0.09,0.175)
ax2.set_title('(b) Mean and confidence bound for LUCB')



history = pd.DataFrame(columns=['Arm'], data=[arm.identifier for arm in BS_LUCB.history])
history = history.merge(pd.get_dummies(history['Arm']), left_index=True, right_index=True)
for arm in list(history['Arm'].unique()):
    history['{}_cumsum'.format(arm)] = history[arm].cumsum()
ax3.stackplot(history.index, history[['1_cumsum','2_cumsum','0_cumsum','3_cumsum']].transpose().values,
              colors=[colours[1], colours[2], colours[0], colours[3]])
ax3.set_xlim(0,max_pulls)
ax3.vlines(initial_pulls*len(BS_LUCB.arms), 0, 100000, linestyle='--')
ax3.text((initial_pulls*len(BS_LUCB.arms))*1.1, 80000, 'initial_pulls done')
ax3.set_title('(c) History for Bootstrapped LUCB')
ax3.set_ylim(0,100000)

history = pd.DataFrame(columns=['Arm'], data=[arm.identifier for arm in LUCB.history])
history = history.merge(pd.get_dummies(history['Arm']), left_index=True, right_index=True)
for arm in list(history['Arm'].unique()):
    history['{}_cumsum'.format(arm)] = history[arm].cumsum()
ax4.stackplot(history.index, history[['1_cumsum','2_cumsum','0_cumsum','3_cumsum']].transpose().values,
             colors=[colours[1], colours[2], colours[0], colours[3]])
ax4.set_xlim(0,max_pulls)
ax4.set_title('(d) History for LUCB')
ax4.set_ylim(0,100000)

for ax in ax1, ax2, ax3, ax4: 
    ax.set_xlabel('pull number')
    
for ax in ax1, ax2:
    ax.set_ylabel('mean reward')
    
for ax in ax3, ax4:
    ax.set_ylabel('cumulative number of pulls')
    
box = ax4.get_position()
ax4.set_position([box.x0, box.y0, box.width * 0.8, box.height])

# Put a legend to the right of the current axis
lgd = fig.legend(loc='center left', bbox_to_anchor=(1, 0.5))

#fig.legend()
plt.tight_layout()
if save_fig: fig.savefig(r"C:\Building Blocks\Innovation - Documents\129. Thesis Jop\Thesis Jop Heuvelmans - BB & JADS\Figures\LUCB_example_2.pdf", bbox_extra_artists=(lgd,), bbox_inches='tight')
plt.show()


history = [arm.identifier for arm in BS_LUCB.history]
crossover = len(history) - 1 - history[::-1].index(0)

fig, ax1 = plt.subplots(figsize=(10,7))
for index, arm in enumerate(BS_LUCB.arms[:3]):
    ax1.plot([n_pulls for n_pulls, value in arm.mean_history] + [BS_LUCB.total_pulls],
            [value for n_pulls, value in arm.mean_history] + [arm.mean_history[-1][1]],
             color = colours[index],
            label='mean arm {}'.format(arm_names[arm.identifier]))
    ax1.fill_between([n_pulls for n_pulls, value in arm.mean_history] + [BS_LUCB.total_pulls],
                     [value for n_pulls, value in arm.LCB_history] + [arm.LCB_history[-1][1]],
                     [value for n_pulls, value in arm.UCB_history] + [arm.UCB_history[-1][1]],
                     alpha=0.5,
                     color = colours[index])
    ax1.hlines(arm.true_mean, 0,BS_LUCB.total_pulls, linestyle=':', color = colours[index], label='true success probability arm {}'.format(arm_names[arm.identifier]))
ylim = (0.105, 0.14)
ax1.vlines(initial_pulls*len(BS_LUCB.arms), ymin = ylim[0], ymax=ylim[1], linestyle='--', lw=1)
ax1.text((initial_pulls*len(BS_LUCB.arms))*1.05, ylim[1]*0.98, 'initial_pulls done')
ax1.vlines(crossover, 0.09,0.175, linestyle='--', lw=1)
ax1.text(crossover*1.01, ylim[1]*0.98, 'control arm (A) beaten')
ax1.set_xlim(0,BS_LUCB.total_pulls + 1000)
ax1.set_ylabel('mean reward')
ax1.set_xlabel('pull number')
ax1.set_title('(a) Mean and CB for Bootstrapped LUCB')

ax1.set_ylim(ylim)
fig.legend(loc='center left', bbox_to_anchor=(1, 0.5))


#fig.legend()
plt.tight_layout()
if save_fig: plt.savefig(r"C:\Building Blocks\Innovation - Documents\129. Thesis Jop\Thesis Jop Heuvelmans - BB & JADS\Figures\LUCB_example_3.pdf")
plt.show()


#### BS_LUCB = meta_experiment.simulations[0].experiments[0]
LUCB = meta_experiment.simulations[1].experiments[0]

width = 0.25
plt.figure(figsize=(8,6))
plt.bar(np.arange(len(BS_LUCB.arms))-width, [arm.true_mean for arm in BS_LUCB.arms], width=width, label='True mean',
       color='#bcbd22')
plt.bar(np.arange(len(BS_LUCB.arms)), [arm.mean for arm in BS_LUCB.arms], width=width, 
        label='Bootstrapped LUCB (after {} pulls)'.format(BS_LUCB.total_pulls),
        yerr = [(arm.UCB - arm.LCB)/2 for arm in BS_LUCB.arms], color='#1f77b4')
plt.bar(np.arange(len(LUCB.arms))+width, [arm.mean for arm in LUCB.arms], width=width,
        label='LUCB (after {} pulls)'.format(LUCB.total_pulls),
        yerr = [(arm.UCB - arm.LCB)/2 for arm in LUCB.arms], color='#d62728')
plt.ylim(0.08, 0.16)
plt.legend()
plt.xlabel('Arm')
plt.ylabel('Mean reward')
plt.xticks(np.arange(len(BS_LUCB.arms)), ('A', 'B', 'C', 'D'))
if save_fig: plt.savefig(r"C:\Building Blocks\Innovation - Documents\129. Thesis Jop\Thesis Jop Heuvelmans - BB & JADS\Figures\LUCB_example.pdf")
plt.show()